In [2]:
from collections import Counter
from datetime import datetime
 
import json
 
from keras.layers import Embedding, LSTM, Dense, Conv1D, MaxPooling1D, Dropout, Activation
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
 
import numpy as np

t1 = datetime.now()
with open("yelp_data/dataset/review.json") as f:
    reviews = f.read().strip().split("\n")
reviews = [json.loads(review) for review in reviews]
print(datetime.now() - t1)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


0:01:49.260744


In [3]:
texts = [review['text'] for review in reviews]

numStars = [review['stars'] for review in reviews]
balanced_texts = []
balanced_labels = []
limit = 100000  # Change this to grow/shrink the dataset
pos_counts = [0, 0, 0, 0, 0]
for i in range(len(texts)):
    score = numStars[i]-1
    if pos_counts[score] < limit:
        balanced_texts.append(texts[i])
        balanced_labels.append(numStars[i]-1)
        pos_counts[score] += 1

In [4]:
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(balanced_texts)
sequences = tokenizer.texts_to_sequences(balanced_texts)
data = pad_sequences(sequences, maxlen=300)

In [5]:
from keras.utils import to_categorical

model = Sequential()
model.add(Embedding(20000, 128, input_length=300))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(5, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'],)
newLabels=np.array(balanced_labels)
newLabels=to_categorical(newLabels)
print(newLabels.shape)
print(data.shape)

(500000, 5)
(500000, 300)


In [6]:
history_big=model.fit(data, newLabels, validation_split=0.4, epochs=10)

Train on 300000 samples, validate on 200000 samples
Epoch 1/10
300000/300000 [==============================] - 8088s 27ms/step - loss: 0.9028 - acc: 0.6132 - val_loss: 1.0729 - val_acc: 0.5262
Epoch 2/10
300000/300000 [==============================] - 8091s 27ms/step - loss: 0.7725 - acc: 0.6692 - val_loss: 1.0729 - val_acc: 0.5451
Epoch 3/10
300000/300000 [==============================] - 8053s 27ms/step - loss: 0.7375 - acc: 0.6848 - val_loss: 1.0438 - val_acc: 0.5514
Epoch 4/10
300000/300000 [==============================] - 6203s 21ms/step - loss: 0.7155 - acc: 0.6960 - val_loss: 1.0392 - val_acc: 0.5519
Epoch 5/10
300000/300000 [==============================] - 4395s 15ms/step - loss: 0.7011 - acc: 0.7039 - val_loss: 1.0099 - val_acc: 0.5716
Epoch 6/10
300000/300000 [==============================] - 4390s 15ms/step - loss: 0.6861 - acc: 0.7125 - val_loss: 1.0213 - val_acc: 0.5697
Epoch 7/10
300000/300000 [==============================] - 4424s 15ms/step - loss: 0.6783 - acc